# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)
city_data_df.set_index("City_ID", inplace=True)
# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,hamilton,39.1834,-84.5333,19.97,86,75,2.57,US,1693382649
1,uturoa,-16.7333,-151.4333,24.97,84,89,9.03,PF,1693382649
2,adamstown,-25.0660,-130.1015,18.27,90,100,3.82,PN,1693382650
3,thunder bay,48.4001,-89.3168,7.99,79,0,1.71,CA,1693382650
4,papao,-17.7500,-149.5500,24.82,89,100,8.38,PF,1693382651


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, size="Humidity", color="City", scale=0.8, tiles="OSM")

# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 27
min_temp = 20
max_wind = 3
max_humid = 80
min_humid = 50

temp_criteria = (city_data_df["Max Temp"] < max_temp) & (city_data_df["Max Temp"] > min_temp)
wind_criteria = city_data_df["Wind Speed"] < max_wind
humid_criteria = (city_data_df["Humidity"] < max_humid) & (city_data_df["Humidity"] > min_humid)

# Drop any rows with null values
filtered_conditions = city_data_df[temp_criteria & wind_criteria & humid_criteria].dropna()

# Display sample data
filtered_conditions.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
15,prainha,-1.8000,-53.4800,22.83,73,0,1.41,BR,1693382656
95,guerrero negro,27.9769,-114.0611,23.71,72,70,2.46,MX,1693382695
99,port douglas,-16.4833,145.4667,21.83,70,12,2.70,AU,1693382697
101,amga,60.8953,131.9608,25.17,52,4,1.75,RU,1693382699
103,tommot,58.9564,126.2925,25.37,51,4,0.45,RU,1693382700


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_conditions.copy()
selected_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = hotel_df[selected_columns]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
15,prainha,BR,-1.8000,-53.4800,73,
95,guerrero negro,MX,27.9769,-114.0611,72,
99,port douglas,AU,-16.4833,145.4667,70,
101,amga,RU,60.8953,131.9608,52,
103,tommot,RU,58.9564,126.2925,51,
133,zhongshu,CN,24.5167,103.7667,79,
137,muhoroni,KE,-0.1541,35.1988,59,
150,fort saskatchewan,CA,53.7168,-113.2187,73,
173,san borja,BO,-14.8167,-66.8500,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
prainha - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
port douglas - nearest hotel: No hotel found
amga - nearest hotel: No hotel found
tommot - nearest hotel: No hotel found
zhongshu - nearest hotel: No hotel found
muhoroni - nearest hotel: No hotel found
fort saskatchewan - nearest hotel: No hotel found
san borja - nearest hotel: No hotel found
floriano - nearest hotel: No hotel found
gurupi - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
khandyga - nearest hotel: No hotel found
bangassou - nearest hotel: No hotel found
itaituba - nearest hotel: No hotel found
vila franca do campo - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
chokwe - nearest hotel: No hotel found
bulgan - nearest hotel: No hotel found
tyukhtet - nearest hotel: No hotel found
bowen - nearest hotel: No hotel found
lutsk - nearest hotel: No hotel found
pesaro - nearest hotel: No hotel found
jaru - nearest hotel: No ho

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
15,prainha,BR,-1.8000,-53.4800,73,No hotel found
95,guerrero negro,MX,27.9769,-114.0611,72,No hotel found
99,port douglas,AU,-16.4833,145.4667,70,No hotel found
101,amga,RU,60.8953,131.9608,52,No hotel found
103,tommot,RU,58.9564,126.2925,51,No hotel found
133,zhongshu,CN,24.5167,103.7667,79,No hotel found
137,muhoroni,KE,-0.1541,35.1988,59,No hotel found
150,fort saskatchewan,CA,53.7168,-113.2187,73,No hotel found
173,san borja,BO,-14.8167,-66.8500,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display


# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    size = "Humidity",
    scale = 5,
    alpha = 0.8,
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    hover_cols = ['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)